In [1]:
from utils import *
import os

ModuleNotFoundError: No module named 'mne_connectivity'

### Define parameters

In [ ]:
# clean up loading 
subj = 'sub-P019'
session = 'ses-S001'
task = 'task-Default'
run = 'run-003'
subject_folder = "../DSI/data/" + subj + "/" + session + "/" #+sub-P019/ses-S001/"

fname = subject_folder + "eeg/" + subj + "_" + session + "_" + task + "_" + run + "_eeg.xdf"
save_path = subject_folder + "features/" + subj + "_" + session + "_" + task + "_" + run + ".csv"

eeg_data = os.path.join(fname)
feature_folder = os.path.join(subject_folder, "features")
if not os.path.exists(feature_folder):
    os.makedirs(feature_folder)

# fname = "../DSI/data/sub-P019/ses-S001/eeg/sub-P019_ses-S001_task-Default_run-003_eeg.xdf"
event_dict = {
            'unknown?': 0,
            'test': 99,
            'start': 88,
            'baseline' : 77, 

            # elbow flexion, elbow extension, forearm supination, forearm pronation, hand close, and hand open
            'elbow_flexion' : 1,
            'elbow_extension' : 2,
            'forearm_supination' : 3,
            'forearm_pronation' : 4,
            'hand_close' : 5,
            'hand_open' : 6
            }
# adjust as you need
freqs = np.arange(1., 40., 1.)  
freq_bins = [(1, 4), (4, 8), (8, 12), (12, 35), (35, 40)]

### Cleaning and Obtaining the Dataset 

In [ ]:
# Create the MNE object
raw, events, fs = build_mne_object(fname, eeg_stream_name='Gwennie-24', stimulus_stream = 'stimulus_stream')
# Set montage
sample_1020 = set_1020_montage(raw)
# Preprocess the data
iir_params = dict(order=50, ftype='butter')  # Higher order = steeper
sample_1020.filter(l_freq=1., h_freq=45., method='iir', iir_params=iir_params)
# Epoching
epochs = mne.Epochs(sample_1020, events, 
                    event_id = event_dict, 
                    baseline=(0,0),
                    tmin=0, 
                    tmax=3, 
                    preload=True,  
                    #reject=reject_criteria,   
                    on_missing='ignore'  # Ignores missing events instead of raising an error
)
# Create dictionary of averaged epochs
epoch_dict = {}
for event in event_dict:
    epoch_dict[event] = epochs[event].average()

# Create daframe for frequency, corr and temporal features
df_power = create_power_df(epoch_dict, freqs, freq_bins = freq_bins)
df_corr, _ = create_corr_df(epoch_dict)
df_temp = create_temp_df(epoch_dict, fs)

# Create final dataframe
dataset = create_final_df(
                          power_df=df_power,
                          fc_df=df_corr,
                          mstd_df=df_temp)

In [ ]:
dataset 

unknown?  \
freq_features Fp1  [[3.1491826331581946e-09, 3.0879550064966325e-...   
              Fp2  [[3.173780596905849e-09, 1.8652507432323087e-0...   
              F7   [[3.2401825532564213e-09, 5.580624490383677e-0...   
              F3   [[1.2972877707687107e-09, 6.536357177295257e-0...   
              Fz   [[2.3849929523381713e-09, 4.629290892081095e-0...   
...                                                              ...   
mstd_features Pz   [[-4.5597008154817735e-06, -4.125597590916775e...   
              P4   [[-8.284927367572934e-06, -2.0684481077446048e...   
              T6   [[5.132263138548135e-06, 1.0060186592977946e-0...   
              O1   [[-8.528338222046058e-06, -1.0125781888764762e...   
              O2   [[-2.532166248156286e-06, -9.817280431747767e-...   

                                                                test  \
freq_features Fp1  [[1.5205670043398545e-08, 9.606852843557792e-0...   
              Fp2  [[8.377160816848214e-08, 3.8822107865653914e-0...   
              F7   [[1.4581750441192945e-08, 1.1597921732309895e-...   
              F3   [[2.0806144014055378e-08, 1.3711274213693217e-...   
              Fz   [[2.6517254026725163e-08, 1.3914056504705135e-...   
...                                                              ...   
mstd_features Pz   [[-1.363813479399089e-05, -6.001993642200053e-...   
              P4   [[-1.0791730352880456e-05, -5.305877709841753e...   
              T6   [[1.8507188707289504e-05, 5.6973420834807084e-...   
              O1   [[-1.4420300700631786e-05, -5.722807316167993e...   
              O2   [[2.5000914848179946e-06, 3.899085509564415e-0...   

                                                               start  \
freq_features Fp1  [[5.1931034087439523e-08, 1.950601475968281e-0...   
              Fp2  [[3.1122145158795592e-09, 8.356671433204515e-0...   
              F7   [[2.439510034884501e-08, 1.2427545900701095e-0...   
              F3   [[1.687963442123637e-08, 1.768877122014862e-08...   
              Fz   [[2.2742730734753308e-08, 1.4577789381563735e-...   
...                                                              ...   
mstd_features Pz   [[-1.6587905895340003e-05, -1.9005877685217844...   
              P4   [[-3.0083153984996693e-05, -1.9811303213563643...   
              T6   [[2.2500865402717164e-05, 4.597907317560438e-0...   
              O1   [[-3.1283024424598356e-05, -1.7481115271371144...   
              O2   [[1.62487970192552e-05, 3.363583631551929e-05,...   

                                                            baseline  \
freq_features Fp1  [[2.2814172594976568e-10, 8.390948006682153e-1...   
              Fp2  [[1.116346567546226e-10, 1.4264881707793502e-1...   
              F7   [[2.9373072855813354e-10, 9.835776040029004e-1...   
              F3   [[3.945604992215883e-10, 3.7386829443081663e-1...   
              Fz   [[5.08389184236757e-10, 4.590720386324783e-10,...   
...                                                              ...   
mstd_features Pz   [[-6.7342556793526155e-06, -8.777592415232763e...   
              P4   [[-7.4249708354319125e-06, -8.970680295545142e...   
              T6   [[-5.400907029564006e-06, -2.1292583399872417e...   
              O1   [[-4.979590095754004e-06, 2.786186238059091e-0...   
              O2   [[-5.01203540908514e-06, -3.996167501044654e-0...   

                                                       elbow_flexion  \
freq_features Fp1  [[4.717841473714235e-10, 5.795772955682969e-10...   
              Fp2  [[3.327403756651793e-10, 3.909190190142297e-10...   
              F7   [[4.5367227129738613e-10, 7.092098661135457e-1...   
              F3   [[2.1357138840293843e-09, 1.157336937979536e-0...   
              Fz   [[3.52137450775874e-10, 2.1756629979938305e-10...   
...                                                              ...   
mstd_features Pz   [[3.862721157455721e-06, 3.1010845930995973e-0...   
              P4   [[1.4

In [ ]:
# save the dataset
dataset.to_csv(save_path, index=False)

In [ ]:
epochs_mov = epochs[['elbow_extension', 'elbow_flexion' , 'forearm_supination', 'forearm_pronation', 'hand_close', 'hand_open']].get_data()
epochs_baseline = epochs[['baseline']].get_data()

mean_mov = np.mean(epochs_mov, axis=0)
mean_baseline = np.mean(epochs_baseline, axis=0)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.renderers.default = "browser"

# Time axis
time = np.arange(mean_mov.shape[1])  # 901 time points

# Create subplots: 19 rows, 1 column
fig = make_subplots(
    rows=19, cols=1,
    shared_xaxes=True,
    vertical_spacing=0.02,
    subplot_titles=[f'Electrode {i+1}' for i in range(19)]
)

# Add traces
for i in range(19):
    fig.add_trace(go.Scatter(
        x=time,
        y=mean_mov[i],
        mode='lines',
        name='Movement' if i == 0 else None,  # Show legend only once
        line=dict(color='blue')
    ), row=i+1, col=1)

    fig.add_trace(go.Scatter(
        x=time,
        y=mean_baseline[i],
        mode='lines',
        name='Baseline' if i == 0 else None,
        line=dict(color='red')
    ), row=i+1, col=1)

# Update layout
fig.update_layout(
    height=4000,  # Adjust as needed
    width=1000,
    title_text="Mean EEG Epochs per Electrode (Movement vs Baseline)",
    showlegend=True
)

fig.update_xaxes(title_text="Time (samples)", row=19, col=1)
fig.update_yaxes(title_text="Amplitude (µV)")

fig.show()
